In [17]:
import numpy as np
import os  
from PIL import Image
import shutil
import sys

try:
    import cv2
    from cv2 import imread,imwrite,imshow
except ImportError:
    # If no OpenCV，then skimage
    import skimage.io as io
    from skimage.io import imread, imsave,imshow
    imwrite = imsave


In [21]:
#global path
#sys.path.append("..")
abs_path = os.path.abspath("__file__"+"/../../data_set/")
original_veg_path=os.path.join(abs_path,"arcgis_datasets/labels/")
original_image_path=os.path.join(abs_path,"arcgis_datasets/images/")

target_veg_path=os.path.join(abs_path,"arcgis_datasets/labels_balance/")
target_image_path=os.path.join(abs_path,"arcgis_datasets/images_balance/")

#train:test:validatio=6:2:2
# target_label_path=os.path.join(abs_path,"train/labels/train/")
# target_image_path=os.path.join(abs_path,"train/images/train/")
# target_label_path=os.path.join(abs_path,"test/labels/test/")
# target_image_path=os.path.join(abs_path,"test/images/test/")
# target_label_path=os.path.join(abs_path,"validation/labels/validation/")
# target_image_path=os.path.join(abs_path,"validation/images/validation/")

In [19]:
id2code={1:(220,245,255),2:(180,245,255),3:(130,245,255),4:(56,245,255),5:(163,255,115),
            6:(85,255,0),7:(215,243,172),8:(85,127,0),9:(144,173,49),10:(161,255,167),
            11:(151,230,1),12:(56,168,2),13:(85,212,89),14:(26,197,30),15:(93,255,133),
            16:(1,230,169),17:(93,255,207),18:(109,190,174),19:(85,190,174),20:(107,191,199),
            21:(178,178,178),22:(255,0,0),23:(254,234,190),24:(214,133,137),25:(255,127,127),
            26:(3,197,255),27:(3,181,245),28:(3,167,255),29:(3,137,255),30:(3,107,255),
            31:(3,77,255),32:(3,47,255),33:(3,17,255),34:(0,77,138),35:(0,77,168),
            36:(0,77,198),37:(0,77,238),38:(255,0,197),39:(236,96,167),40:(255,125,197),
            41:(185,142,0),42:(180,171,93),43:(0,0,0)}

In [22]:
def rgb_to_onehot(rgb_image, colormap=id2code):
    """
    Function to one hot encode RGB mask labels.
    Args:
        rgb_image: image matrix (eg. 256 x 256 x 3 dimension numpy ndarray).
        colormap: dictionary of color to original_label id.
    Return:
        encoded_image: One hot encoded image of dimensions (height x width x num_classes) where num_classes = len(colormap).
    Raises:
        None.
    """
    shape=43
    encoded_image = np.zeros(shape, dtype=np.int8)
    for i, cls in enumerate(colormap):
        encoded_image[:, :, i] = np.all(rgb_image.reshape((-1, 3)) == colormap[cls], axis=1).reshape(shape[:2])
    return encoded_image

In [23]:
#initial indicators
flag=np.zeros(43,dtype=np.uint)
pixel_total=np.zeros(43,dtype=np.uint)

# how many titles(purity) for each 43 classifiction.
pic_max=np.full(43,fill_value=300, dtype=np.uint) 
#pic_max=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
pixel_max=pic_max * 256 *256

#labels
for file in os.listdir(original_veg_path):
    filename = os.fsdecode(file)
    # read pic and get shape size
    original_veg_path1=os.path.join(original_veg_path, filename)   
    img = imread(original_veg_path1)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    #imshow('',img)
    
    shape = img.shape[:2] + (43,)
    encoded_image = np.zeros(shape, dtype=np.int8)

    for i, cls in enumerate(id2code):
        encoded_image[:, :, i] = np.all(img.reshape((-1, 3)) == id2code[cls], axis=1).reshape(shape[:2])

    if (encoded_image[:,:]*flag).sum()==0:

        #move this picture
        shutil.copyfile(os.path.join(original_veg_path,filename), os.path.join(target_veg_path,filename))
        shutil.copyfile(os.path.join(original_image_path,str.replace(filename,"veg1","image"))
                        ,os.path.join(target_image_path,str.replace(filename,"veg1","image")))
        
        one_pixel_total=np.sum(np.sum(encoded_image[:,:],axis=1),axis=0)# each classifiction total pixels of each picture
        pixel_total=np.sum([pixel_total,one_pixel_total],axis=0)
        #update flag
        flag=np.multiply((pixel_total>=pixel_max),1)

    if  flag.sum()==43:
        break
        
print(flag)
print(pixel_total)
    
    
        


[0 0 0 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1]
[  600468.  4026120.        0.        0. 19694448. 19699908. 19695220.
 19676788.   935464. 19699640.  3308064. 19706624.  4475256. 19670476.
  1892008. 19661556.        0.        0. 19699116.        0. 19680008.
  2396564. 19665796.        0.  6394140. 15012512.  1667196.        0.
        0.  4992540.        0.  1925676.        0.  1556924.        0.
    49932.  2011492.  6797928.        0.        0.        0. 19683244.
 19695832.]


In [24]:
pixel_total/(256*256)

array([  9.16241455,  61.43371582,   0.        ,   0.        ,
       300.51342773, 300.59674072, 300.52520752, 300.24395752,
        14.27404785, 300.59265137,  50.47705078, 300.69921875,
        68.2869873 , 300.14764404,  28.86975098, 300.01153564,
         0.        ,   0.        , 300.58465576,   0.        ,
       300.29309082,  36.56866455, 300.07623291,   0.        ,
        97.5668335 , 229.07275391,  25.43939209,   0.        ,
         0.        ,  76.18011475,   0.        ,  29.38348389,
         0.        ,  23.7567749 ,   0.        ,   0.76190186,
        30.69293213, 103.72814941,   0.        ,   0.        ,
         0.        , 300.34246826, 300.5345459 ])